## Series
Series是pandas系列里的一维数组，它可以包含任何数据类型的标签。我们主要使用它们来处理时间序列数据。

### 修改索引

创造一个series,获取series的名称。

In [ ]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, np.nan, 4, 5])
print(s)
print(s.name)

添加名称，获取索引。

In [ ]:
s.name = "Price Series"
print(s.name)
print(s.index)

修改索引

In [ ]:
new_index = pd.date_range("20160101",periods=len(s), freq="D")
s.index = new_index
print (s.index)

### 访问系列元素

系列的访问通常使用iloc[]和loc[]的方法。我们使用iloc[]来访问元素的整数索引和我们使用loc[]来访问指数序列的。

访问单个整数索引：

In [ ]:
print("First element of the series: ", s.iloc[0])
print("Last element of the series: ", s.iloc[len(s)-1])

访问范围的整数索引，从0到5，间隔2：

In [ ]:
print(s.iloc[0:5:2])

访问单个与范围的指数序列：

In [ ]:
print(s.loc['20160101'])
print(s.loc['20160102':'20160104'])

### 布尔索引

除了上述访问方法,您可以使用布尔过滤序列数组。比较序列与标准是否一致。当与您设定的任何条件相比,这次你返回另一个系列中,回填满了布尔值。

In [ ]:
print(s < 3)
print(s.loc[s < 3])
print(s.loc[(s < 3) & (s > 1)])

### 索引和时间序列

因为我们使用Series处理时间序列,值得涵盖一点是我们如何处理时间组件。对于我们的目的,我们使用pandas时间戳的对象。通过excel获取数据,进行resample()、data_range()、reindex()。

In [ ]:
import pandas as pd

data = pd.read_excel('sz50.xlsx', sheetname=0, index_col='datetime')
print(data.index)

只保留data中的close，获取data的数据类型与前5个值：

In [ ]:
Series = data.close

In [ ]:
Series.head()

用resample给每个月的最后一天抽样。

In [ ]:
monthly_prices = Series.resample('M').last()
print(monthly_prices.head(5))

In [ ]:
monthly_prices_med = Series.resample('M').median()
monthly_prices_med.head(10)

用月初的值来替代月底的值：

In [ ]:
def custom_resampler(array_like):
    """ Returns the first value of the period """
    return array_like[0]
first_of_month_prices = Series.resample('M').apply(custom_resampler)
first_of_month_prices.head(5)

### 缺失数据

当我们处理实际数据,有一个非常现实的遭遇缺失值的可能性。真实数据充斥着洞和pandas提供我们处理它们的方法。有时重新取样或重建索引可以创建NaN值。 
幸运的是,pandas为我们提供了处理它们的方法。我们有两个处理缺失数据的主要手段，一个是fillna,另一个是dropna

In [ ]:
from datetime import datetime

data_s= Series.loc[datetime(2017,1,1):datetime(2017,1,10)]
data_r=data_s.resample('D').mean() #插入每一天
print(data_r.head(10))

第一个是填写缺失的数据与fillna()。例如,假设我们想填写缺失的天为前一天的价格。

In [ ]:
print(data_r.head(10).fillna(method='ffill'))

我们的下一个选择是明显更有吸引力。我们可以简单地把缺失的数据使用dropna()方法。这是比填充任意数字到NaN值是更好的选择。

In [ ]:
print(data_r.head(10).dropna())

### 用pandas做时间序列分析

让我们对最初的价格做一些基本的时间序列分析。每个Pandas的series都有一个内置命令绘制方法。

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 7))
plt.plot(Series)
plt.title("Stock Prices")
plt.ylabel("Price")
plt.xlabel("Date")
plt.show()

print("Mean: ", data.mean())
print("Standard deviation: ", data.std())
print("Summary Statistics", data.describe())

In [ ]:
add_returns = Series.diff()[1:]
mult_returns = Series.pct_change()[1:]

plt.figure(figsize=(15, 7))
plt.title("returns of Stock")
plt.xlabel("Date")
plt.ylabel("Percent Returns")
mult_returns.plot()
plt.show()

In [ ]:
rolling_mean = Series.rolling(window=40).mean()
rolling_mean.name = "40day rolling mean"

plt.figure(figsize=(15, 7))
Series.plot()
rolling_mean.plot()
plt.title("Stock Price")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

In [ ]:
rolling_std = Series.rolling(window=40).std()
rolling_std.name = "40day rolling volatility"

plt.figure(figsize=(15, 7))
rolling_std.plot()
plt.title(rolling_std.name)
plt.xlabel("Date")
plt.ylabel("Standard Deviation")
plt.show()